In [3]:
# Importing the relevant libraries
import IPython.display
import json
import pandas as pd
import seaborn as sns
#import squarify
%matplotlib inline
import random
#import missingno as msno
#import plotly.offline as py
#py.init_notebook_mode(connected=True)
#import plotly.graph_objs as go
#import plotly.tools as tls
import numpy as np
from matplotlib import pyplot as plt

# D3 modules
from IPython.core.display import display, HTML, Javascript
from string import Template# The data to load

# Data Import

Because the training data is so large (125 Million rows), we will begin by taking a random sample of the rows to have a valid representative sample. 

In [5]:
%%time

f = "Data/train.csv"

# Count the lines
num_lines = sum(1 for l in open(f))

# Sample size - in this case ~10%
size = int(num_lines / 20)

# The row indices to skip - make sure 0 is not included to keep the header!
skip_idx = random.sample(range(1, num_lines), num_lines - size)

# Read the data
train = pd.read_csv(f, skiprows=skip_idx, parse_dates=['date'] )

<string>:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 5min 26s


The remaining files load quickly. 

In [52]:
items = pd.read_csv("Data/items.csv")
holiday_events = pd.read_csv("Data/holidays_events.csv",parse_dates=['date'])
stores = pd.read_csv("Data/stores.csv")
oil = pd.read_csv("Data/oil.csv",parse_dates=['date'])
transactions = pd.read_csv("Data/transactions.csv",parse_dates=['date'])
test = pd.read_csv("Data/test.csv",parse_dates=['date'])

# Checking for Null Values

A good first step is to see which columns in which dataframes contain null values. If these appear, we will need to deal with these in some way. 

In [15]:
print("Nulls in train columns: {0} => {1}".format(train.columns.values, train.isnull().any().values))

Nulls in train columns: ['id' 'date' 'store_nbr' 'item_nbr' 'unit_sales' 'onpromotion'] => [False False False False False  True]


The onpromotion column of the train dataframe contains null values.

In [17]:
print("Nulls in test columns: {0} => {1}".format(test.columns.values, test.isnull().any().values))

Nulls in test columns: ['id' 'date' 'store_nbr' 'item_nbr' 'onpromotion'] => [False False False False False]


In [18]:
print("Nulls in items columns: {0} => {1}".format(items.columns.values, items.isnull().any().values))

Nulls in items columns: ['item_nbr' 'family' 'class' 'perishable'] => [False False False False]


In [19]:
print("Nulls in holiday_events columns: {0} => {1}".format(holiday_events.columns.values, holiday_events.isnull().any().values))

Nulls in holiday_events columns: ['date' 'type' 'locale' 'locale_name' 'description' 'transferred'] => [False False False False False False]


In [14]:
print("Nulls in stores columns: {0} => {1}".format(stores.columns.values, stores.isnull().any().values))

Nulls in stores columns: ['store_nbr' 'city' 'state' 'type' 'cluster'] => [False False False False False]


In [20]:
print("Nulls in oil columns: {0} => {1}".format(oil.columns.values, oil.isnull().any().values))

Nulls in oil columns: ['date' 'dcoilwtico'] => [False  True]


In [21]:
print("Nulls in transactions columns: {0} => {1}".format(transactions.columns.values, transactions.isnull().any().values))

Nulls in transactions columns: ['date' 'store_nbr' 'transactions'] => [False False False]


It looks like there are two columns that contain null values. One is the onpromotion column of the train data, and the other is the dcoilwtico column of the oil price data. 

# Dealing with Null Values

The first column that contains null values is the onpromotion column of the train data. My initial thought was to simply change all the null values to false. However, this has the potential to mislead predictions. Instead, I followed the suggestion on the website provided to set missing values to 2, True to 1, and False to 0. 

Credit goes to: http://kevincsong.com/Kaggle-Pt.1-Favorita-Grocery-Sales-Prediction-Data-Engineering/

In [53]:
train.loc[:, 'onpromotion'].fillna(2, inplace=True) # Replaces null values with 2

In [54]:
train.loc[:, 'onpromotion'].replace(True, 1, inplace=True) # Replace True values with 1

In [55]:
train.loc[:, 'onpromotion'].replace(False, 0, inplace=True) # Replace False values with 0

It makes sense to make analogous changes in the test file.

In [56]:
test.loc[:, 'onpromotion'].fillna(2, inplace=True) # Replaces null values with 2

In [57]:
test.loc[:, 'onpromotion'].replace(True, 1, inplace=True) # Replace True values with 1

In [58]:
test.loc[:, 'onpromotion'].replace(False, 0, inplace=True) # Replace False values with 0

Check to see if the changes occurred as expected.

In [59]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,dow,doy
0,25,2013-01-01,25,129635,11.0,2,1,1
1,70,2013-01-01,25,222879,5.0,2,1,1
2,71,2013-01-01,25,223434,1.0,2,1,1
3,85,2013-01-01,25,261053,1.0,2,1,1
4,98,2013-01-01,25,273528,2.0,2,1,1


In [60]:
test.head()

,id,date,store_nbr,item_nbr,onpromotion
0,125497040,2017-08-16,1,96995,0.0
1,125497041,2017-08-16,1,99197,0.0
2,125497042,2017-08-16,1,103501,0.0
3,125497043,2017-08-16,1,103520,0.0
4,125497044,2017-08-16,1,103665,0.0


Success

The second column that contains null values is the dcoilwtico in the oil file. It will probably be best to interpolate the missing values in some way. To start, I looked at the rows that are null. It appears there are only about 43 dates out of 1218 that are missing values. I will go ahead and simply backfill the oil prices. 

In [61]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
date          1218 non-null datetime64[ns]
dcoilwtico    1175 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.1 KB


In [62]:
# Replace NaN values using backward fill to cover first date 
oil['dcoilwtico'].fillna(method='bfill', inplace=True)

In [63]:
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
date          1218 non-null datetime64[ns]
dcoilwtico    1218 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.1 KB


No more null values in the oil price column.

# Add Day of Week and Day of Year

I came across a very clever idea to include the day of the week and day of the year as separate columns in the training data. This way our solution could find patterns regarding the day of the week, which probably plays a huge role in sales. It's expected that sales increase on the weekends. Special thanks to Kevin Song: 
http://kevincsong.com/Kaggle-Pt.1-Favorita-Grocery-Sales-Prediction-Data-Engineering/

In [64]:
# Add 'dow' and 'doy'
train['dow'] = train['date'].dt.dayofweek # adding day of week as a feature
train['doy'] = train['date'].dt.dayofyear # adding day of year as a feature

test['dow'] = test['date'].dt.dayofweek # adding day of week as a feature
test['doy'] = test['date'].dt.dayofyear # adding day of year as a feature

# Merge the Data into a Single DataFrame

In [66]:
train_merged = pd.merge(train, stores, on='store_nbr', how='left')
train_merged = pd.merge(train_merged, items, on='item_nbr', how='left')
train_merged = pd.merge(train_merged, oil, on='date', how='left')
train_merged = pd.merge(train_merged, holiday_events, on='date', how='left')

train_merged.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,dow,doy,city,state,...,cluster,family,class,perishable,dcoilwtico,type_y,locale,locale_name,description,transferred
0,25,2013-01-01,25,129635,11.0,2,1,1,Salinas,Santa Elena,...,1,DAIRY,2112,1,93.14,Holiday,National,Ecuador,Primer dia del ano,False
1,70,2013-01-01,25,222879,5.0,2,1,1,Salinas,Santa Elena,...,1,GROCERY I,1058,0,93.14,Holiday,National,Ecuador,Primer dia del ano,False
2,71,2013-01-01,25,223434,1.0,2,1,1,Salinas,Santa Elena,...,1,GROCERY I,1032,0,93.14,Holiday,National,Ecuador,Primer dia del ano,False
3,85,2013-01-01,25,261053,1.0,2,1,1,Salinas,Santa Elena,...,1,GROCERY I,1072,0,93.14,Holiday,National,Ecuador,Primer dia del ano,False
4,98,2013-01-01,25,273528,2.0,2,1,1,Salinas,Santa Elena,...,1,PERSONAL CARE,4114,0,93.14,Holiday,National,Ecuador,Primer dia del ano,False


In [68]:
for i in train_merged.columns:
    print(i, train_merged[i].isnull().any())

id False
date False
store_nbr False
item_nbr False
unit_sales False
onpromotion False
dow False
doy False
city False
state False
type_x False
cluster False
family False
class False
perishable False
dcoilwtico True
type_y True
locale True
locale_name True
description True
transferred True


# Sampling a Specific Item

Just as a sanity check, I thought it would be a good idea to sample a particular item at a particular location to see what the data looked like. 

In [73]:
PN129635 = train_merged[train_merged['item_nbr'] == 129635]

In [74]:
PN129635 = PN129635[PN129635['store_nbr'] == 25]

In [79]:
PN129635.head(10)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,dow,doy,city,state,...,cluster,family,class,perishable,dcoilwtico,type_y,locale,locale_name,description,transferred
0,25,2013-01-01,25,129635,11.0,2,1,1,Salinas,Santa Elena,...,1,DAIRY,2112,1,93.14,Holiday,National,Ecuador,Primer dia del ano,False
15169,303263,2013-01-09,25,129635,8.0,2,2,9,Salinas,Santa Elena,...,1,DAIRY,2112,1,93.08,NaN,NaN,NaN,NaN,NaN
61598,1231095,2013-02-01,25,129635,3.0,2,4,32,Salinas,Santa Elena,...,1,DAIRY,2112,1,97.46,NaN,NaN,NaN,NaN,NaN
80417,1610080,2013-02-10,25,129635,30.0,2,6,41,Salinas,Santa Elena,...,1,DAIRY,2112,1,NaN,NaN,NaN,NaN,NaN,NaN
148918,2982700,2013-03-15,25,129635,56.0,2,4,74,Salinas,Santa Elena,...,1,DAIRY,2112,1,93.49,NaN,NaN,NaN,NaN,NaN
159745,3199988,2013-03-20,25,129635,4.0,2,2,79,Salinas,Santa Elena,...,1,DAIRY,2112,1,93.21,NaN,NaN,NaN,NaN,NaN
192358,3844012,2013-04-04,25,129635,12.0,2,3,94,Salinas,Santa Elena,...,1,DAIRY,2112,1,93.26,NaN,NaN,NaN,NaN,NaN
203281,4061230,2013-04-09,25,129635,9.0,2,1,99,Salinas,Santa Elena,...,1,DAIRY,2112,1,94.18,NaN,NaN,NaN,NaN,NaN
234944,4700617,2013-04-24,25,129635,2.0,2,2,114,Salinas,Santa Elena,...,1,DAIRY,2112,1,91.07,NaN,NaN,NaN,NaN,NaN
405049,7941984,2013-07-06,25,129635,13.0,2,5,187,Salinas,Santa Elena,...,1,DAIRY,2112,1,NaN,NaN,NaN,NaN,NaN,NaN


One thing that sticks out is that the oil price is NaN whenever the day of the week is 5 or 6. This means that the oil price isn't tracked on the weekends. This makes sense and it will probably be sufficient to forwardfill using Friday oil prices.  

In [80]:
# Replace NaN values using forward fill to cover weekend dates using Friday's oil prices 
train_merged['dcoilwtico'].fillna(method='ffill', inplace=True)